In [ ]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools

In [ ]:
random.seed(1356) # set seed so that each play is the same

In [ ]:
def hex_add(a, b):
    return Hex[a[0] + b[0], a[1] + b[1], a[2] + b[2]]

def hex_subtract(a, b):
    return Hex[a[0] - b[0], a[1] - b[1], a[2] - b[2]]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_directions[direction])

def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex[0]) + abs(hex[1]) + abs(hex[2])) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

In [ ]:
hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

In [ ]:
random.seed(1356)
side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 5 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4

In [ ]:
def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

board = initializeBoard()
board, len(board)

In [ ]:
def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

# wzobristboard

In [ ]:
def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard


# bzobristboard

In [ ]:
def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex


# zboard

In [ ]:
def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [1]:
board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

boardToZobrist(board, zboard, wzobristboard, bzobristboard)

NameError: name 'initializeBoard' is not defined

In [ ]:
def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

In [ ]:
# Form groups
# score = find_scores(board, 'w')
def findGroups(score, board):
    new_list = []
    group = {}
    group_state = {}
    group_count = 0

    for tile in score:
        counter = 0
        group[group_count] = 0
        if not tile in new_list:
            for direction in hex_directions:
                new_tile = tuple(map(sum, zip(tile, direction)))
                if new_tile in score:
                    new_list += [new_tile] # Already seen in group

                if score[tile] == board[direction]:
                    group[group_count] += counter
                    group_state.setdefault(tile, []).append(new_tile)

                counter += 1
        group_count += 1
    return group_state

In [ ]:
# Check if lists can be grouped by at least 1 common member.
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [ ]:
# Count total of each group
# group = findGroups(score, board)

def totalgroups(group):
    count_list = []

    for tile in group:
        for tile2 in group:
            if tile != tile2:
                intersect = list(Intersection(group[tile], group[tile2]))

                if len(intersect) != 0:
                    count_list += [len(list(set(group[tile] + group[tile2])))]
    return count_list


# print(totalgroups(group))

In [ ]:
# board = {
#                  (-1, 0, 1): 'w',
#                  (-1, 1, 0): 'b',
#                  (0, -1, 1): 'b',
#                  (0, 0, 0): '',
#                  (0, 1, -1): 'b',
  
#                  (1, -1, 0): 'b',
#                  (1, 0, -1): 'w'}

# fullBoard(board)

In [ ]:
score = find_scores(board, 'b') #score of player1
score2 = find_scores(board, 'w') #score of player2

group = findGroups(score, board)
group2 = findGroups(score2, board)

count_list = totalgroups(group)
count_list2 = totalgroups(group2)

print(totalgroups(group), totalgroups(group2))

In [ ]:
# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

result(count_list, count_list2)

In [ ]:
# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

k

In [ ]:
k


In [30]:
#Creating entire the game tree for the play
from copy import deepcopy
import time

def buildGameTree(board, depth):
    depth += 1
    gametree = {}
    tempscore = 1
    
    zboard = initializeBoardZobrist() # initial board, without any pieces
    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    for i in range(len(all_cells)):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        
        count = 0
        while not valid:
            if count < 1:
                valid = True
            cell = all_cells[count]
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]
            
            #Make two valid moves
            if validMove(board2, x, y):
                if validMove(board2, x2, y2):
                    makeMove(board2, [x, y], 'b')

                    makeMove(board2, [x2, y2], 'w')
                    valid = True
            
                    # Create a zobrist key for the new board/childnode
                    zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

                    # If the board is full, tell the score
                    if fullBoard(board2):
                        
                        score = find_scores(board2, 'b') #score of player1
                        score2 = find_scores(board2, 'w') #score of player2

                        group = findGroups(score, board2)
                        group2 = findGroups(score2, board2)

                        count_list = totalgroups(group)
                        count_list2 = totalgroups(group2)
                        
                        player_one_total, player_two_total = result(count_list, count_list2)
                        
                        # Did player 1 win, lose or draw?
                        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
                        tempscore = ((['l', -999], ['w', 999])[player_one_total < player_two_total ], ['d', -499])[player_one_total == player_two_total]
                        
                        #Store the final game state
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore[1], {tempscore[0]}])
                        
                    else:
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {}])#childTrees(board2, depth)]) # store full board + temporary score + horizon effect?
            
            count += 1
#             break
        all_cells = all_cells[count:]
    return gametree

# start = time.time()
gametree = buildGameTree(board, 0)#13)
# end = time.time()
# print('Time: ', end - start)
# gametree

In [31]:
def childNodes(gametree, lookUpTable): # pass full tree, MCTS style
#     lookUpTable = {}
    for key in gametree:
        nodelist = random.sample(gametree[key], round(len(gametree[key])*.2)) #select random nodes for mcts
        for node in nodelist:
            
            newBoard = node[1]
            newDepth = node[2]
            
            if not fullBoard(newBoard) :
                newChild = buildGameTree(newBoard, newDepth) 
                node[4] = newChild # Add new children to nodes.
#                 lookUpTable[node[0]] = node[1]
                childNodes(node[4], lookUpTable)
            else:
                break
                
    return gametree, lookUpTable

gametree = buildGameTree(board, 0)
start = time.time()
gametree_dummy, lookUp = childNodes(gametree, {})
end = time.time()
print('Time: ', end - start)

KeyboardInterrupt: 

In [ ]:
gametree_dummy

In [32]:
#Creating entire the game tree for the play
from copy import deepcopy
import time

def buildGameTreeFaster(board, depth, nr_samples):
    depth += 1
    gametree = {}
    tempscore = 1
    
    zboard = initializeBoardZobrist() # initial board, without any pieces
    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    for i in range(nr_samples):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        
        count = 0
        while not valid:
            if count < 1:
                valid = True
            count = random.randint(0, len(all_cells)-1)
            cell = all_cells[count]
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]
            
            #Make two valid moves
            if validMove(board2, x, y):
                if validMove(board2, x2, y2):
                    makeMove(board2, [x, y], 'b')

                    makeMove(board2, [x2, y2], 'w')
                    valid = True
                    
                    all_cells.remove(cell)
            
                    # Create a zobrist key for the new board/childnode
                    zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

                    # If the board is full, tell the score
                    if fullBoard(board2):
                        
                        score = find_scores(board2, 'b') #score of player1
                        score2 = find_scores(board2, 'w') #score of player2

                        group = findGroups(score, board2)
                        group2 = findGroups(score2, board2)

                        count_list = totalgroups(group)
                        count_list2 = totalgroups(group2)
                        
                        player_one_total, player_two_total = result(count_list, count_list2)
                        
                        # Did player 1 win, lose or draw?
                        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
                        tempscore = ((-999, 999)[player_one_total < player_two_total ], -499)[player_one_total == player_two_total]
                        
                        #Store the final game state
                        gametree.setdefault(zidentity0, []).append([zidentity, depth, tempscore, {game_state}])
                        
                    else:
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {}])#childTrees(board2, depth)]) # store full board + temporary score + horizon effect?
            
            count += 1
#             break
#         all_cells = all_cells[count:]
    return gametree

start = time.time()
gametreef = buildGameTreeFaster(board, 0, 5)#13)
end = time.time()
print('Time: ', end - start)
gametreef

Time:  0.011256694793701172


{14024079292171647447: [[8268032979383892007,
   {(-5, 0, 5): 'w',
    (-5, 1, 4): '',
    (-5, 2, 3): '',
    (-5, 3, 2): '',
    (-5, 4, 1): '',
    (-5, 5, 0): '',
    (-4, -1, 5): '',
    (-4, 0, 4): '',
    (-4, 1, 3): '',
    (-4, 2, 2): '',
    (-4, 3, 1): '',
    (-4, 4, 0): '',
    (-4, 5, -1): '',
    (-3, -2, 5): '',
    (-3, -1, 4): '',
    (-3, 0, 3): '',
    (-3, 1, 2): '',
    (-3, 2, 1): '',
    (-3, 3, 0): '',
    (-3, 4, -1): '',
    (-3, 5, -2): '',
    (-2, -3, 5): '',
    (-2, -2, 4): '',
    (-2, -1, 3): '',
    (-2, 0, 2): '',
    (-2, 1, 1): '',
    (-2, 2, 0): '',
    (-2, 3, -1): '',
    (-2, 4, -2): '',
    (-2, 5, -3): '',
    (-1, -4, 5): '',
    (-1, -3, 4): '',
    (-1, -2, 3): '',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): '',
    (-1, 3, -2): '',
    (-1, 4, -3): '',
    (-1, 5, -4): '',
    (0, -5, 5): '',
    (0, -4, 4): '',
    (0, -3, 3): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1)

In [37]:
def childNodesFaster(gametree, lookUpTable): # pass full tree, MCTS style
#     lookUpTable = {}
    for key in gametree:
        nodelist = random.sample(gametree[key], round(len(gametree[key])*1)) #select random nodes for mcts
#         nodelist = gametree[key]
        for node in nodelist:
            
            newBoard = node[1]
            newDepth = node[2]
            
            if not fullBoard(newBoard) :
                newChild = buildGameTreeFaster(newBoard, newDepth, 5) 
                node[4] = newChild # Add new children to nodes.
                childNodesFaster(node[4], lookUpTable)
            else:
                break
                
    return gametree, lookUpTable

start = time.time()
for i in range(1000):
    gametreefr, lookUp = childNodesFaster(gametreef, {})
end = time.time()
print('Time: ', end - start)
gametreefr

Time:  212.342444896698


{14024079292171647447: [[8268032979383892007,
   {(-5, 0, 5): 'w',
    (-5, 1, 4): '',
    (-5, 2, 3): '',
    (-5, 3, 2): '',
    (-5, 4, 1): '',
    (-5, 5, 0): '',
    (-4, -1, 5): '',
    (-4, 0, 4): '',
    (-4, 1, 3): '',
    (-4, 2, 2): '',
    (-4, 3, 1): '',
    (-4, 4, 0): '',
    (-4, 5, -1): '',
    (-3, -2, 5): '',
    (-3, -1, 4): '',
    (-3, 0, 3): '',
    (-3, 1, 2): '',
    (-3, 2, 1): '',
    (-3, 3, 0): '',
    (-3, 4, -1): '',
    (-3, 5, -2): '',
    (-2, -3, 5): '',
    (-2, -2, 4): '',
    (-2, -1, 3): '',
    (-2, 0, 2): '',
    (-2, 1, 1): '',
    (-2, 2, 0): '',
    (-2, 3, -1): '',
    (-2, 4, -2): '',
    (-2, 5, -3): '',
    (-1, -4, 5): '',
    (-1, -3, 4): '',
    (-1, -2, 3): '',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): '',
    (-1, 3, -2): '',
    (-1, 4, -3): '',
    (-1, 5, -4): '',
    (0, -5, 5): '',
    (0, -4, 4): '',
    (0, -3, 3): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1)

In [ ]:
# Create path from root end node, MCTS style
def createPath(gametree, path):
    
    if type(gametree) != set:
        for node in gametree:
            for rnd in range(len(gametree[node])):
                tempPath = []
#                 rnd = random.randint(0, len(gametree[node])-1) # pick random node
                selected_node = gametree[node][rnd]

                path += [(selected_node[0], selected_node[2])] #add node to path
                createPath(selected_node[4], path) # check next node
    else:
        path += [gametree]
    
    return path

start = time.time()
paths = []
for i in range(1000):
    paths += [createPath(gametree, [])]
end = time.time()
print('Time: ', end - start)
    
paths

In [ ]:
# Transposition table
def createTT(paths):
    tt = {}

    for path in paths:
        order = []
        for i in range(len(path)-1):

            if path[i][0] in tt:

                # Update value if, the newnode discover deeper than the one in the tt
                if path[i][1] >= tt[path[i][0]][1]: 
                    
                    powerUp = 1
                    if tt[path[i][0]][0][0] > (tt[path[i][0]][0][1] * 1.3) and \
                        tt[path[i][0]][0][0] > tt[path[i][0]][0][2]:
                        powerUp = 27

                    # When deeper, Reward win-lose-draw extra
                    if path[-1] == {'w'}:
                        tt[path[i][0]][0][0] += path[i][1] + powerUp
                    elif path[-1] == {'l'}:
                        tt[path[i][0]][0][1] += path[i][1] + powerUp
                    elif path[-1] == {'d'}:
                        tt[path[i][0]][0][2] += path[i][1] + powerUp# 


                    # Update depth and next state
                    if type(path[i+1]) != set:
                        tt[path[i][0]][2] = path[i+1][0]
                    else:
                         tt[path[i][0]][2] = path[-1]

                    tt[path[i][0]][1] = path[i][1] # Update depth
                else:
                    # Count number of win-lose-draws for paths
                    if path[-1] == {'w'}:
                        tt[path[i][0]][0][0] += 1
                    elif path[-1] == {'l'}:
                        tt[path[i][0]][0][1] += 1
                    elif path[-1] == {'d'}:
                        tt[path[i][0]][0][2] += 1#
            else:
                if type(path[i+1]) != set:
                    tt[path[i][0]] = [[1, 1, 1], path[i][1], path[i+1][0]] # win-lose-draw, depth, next
                else:
                    tt[path[i][0]] = [[1, 1, 1], path[i][1], path[-1]] # win-lose-draw, depth, gamestae
            
    return tt

tt = createTT(paths)
tt

In [ ]:
from ipywidgets import widgets
from IPython.display import display

text = widgets.Text()
display(text)

def handle_text(sender):
    print(text.value)

text.on_submit(makeMove(board, [0,0], 'b'))

In [ ]:
INFINITY = 999

def minimax(node, depth, isMaximizingPlayer, alpha, beta):
#     print('Node', node, '\n\n\n\n')
    if type(node) is list : #### TODO *
        return node[3] #### TODO *
    
    if isMaximizingPlayer :
        bestVal = -INFINITY
        for key in node:
            for n in node[key]:
                
    #         for n in node: #### TODO * 
                if type(n[4]) != set:
                    value = minimax(n[4], depth+1, False, alpha, beta)
                else:
                    value = minimax(n, depth+1, True, alpha, beta)
                bestVal = max( bestVal, value) 
                alpha = max( alpha, bestVal)
                if beta <= alpha:
                    # remove node from tree
                    break
        return bestVal

    else :
        bestVal = +INFINITY 
        for key in node:
            for n in node[key]:
                
#         for n in node: #### TODO *
                if type(n[4]) != set:
                    value = minimax(n[4], depth+1, True, alpha, beta)
                else:
                    value = minimax(n, depth+1, True, alpha, beta)
                bestVal = min( bestVal, value) 
                beta = min( beta, bestVal)
                if beta <= alpha:
                    # remove node from tree
                    break
        return bestVal
    
minimax(gametree, 0, True, -INFINITY, +INFINITY) #### TODO

In [ ]:
# INFINITY = 9999

# def minimaxTreeBuilder(node, depth, isMaximizingPlayer, alpha, beta):

#     if type(node) is list : #is leaf
#         return node[3] # return leaf value
    
#     if isMaximizingPlayer :
#         bestVal = -INFINITY +7
#         for key in node:
#             for n in node[key]:
                
#                 if type(n[4]) != set:
#                     value = minimaxTreeBuilder(n[4], depth+1, False, alpha, beta)
#                 else:
#                     value = minimaxTreeBuilder(n, depth+1, True, alpha, beta)

#                 bestVal = max( bestVal, value) 
#                 alpha = max( alpha, bestVal)
#                 if beta <= alpha:
#                     # remove node from tree

# #                     del tree
#                     break
#                 else:
#                     n[4] = buildGameTree(n[1], n[3])#, lookUpTable)
#         return bestVal

#     else :
#         bestVal = +INFINITY +7
#         for key in node:
#             for n in node[key]:
                
#                 if type(n[4]) != set:
#                     value = minimax(n[4], depth+1, True, alpha, beta)
#                 else:
#                     value = minimax(n, depth+1, True, alpha, beta)

#                 bestVal = min( bestVal, value) 
#                 beta = min( beta, bestVal)
#                 if beta <= alpha:
#                     # remove node from tree
# #                     del tree
#                     break
#                 else:
#                     print('hi', n[3])
#                     n[4] = buildGameTree(n[1], n[3])#, lookUpTable)
#         return bestVal
    
# minimax(gametree, 0, True, -INFINITY, +INFINITY) #### TODO

In [29]:
INFINITY = 9999

def minimaxTreeBuilder(node, depth, isMaximizingPlayer, alpha, beta, visited): #NegaMinimax
    if type(node) is list:
        print('returned', node[3])
        return node[3]
    
#     if isMaximizingPlayer:
    bestValue = -INFINITY
    for key in node:
        nodes = node[key]
        for i in range(len(nodes)):

            n = node[key][i] # Individual node
            
            if n[0] in visited:
                continue
            else:
                visited += [n[0]]
#             if not fullBoard(n[1]) and not n[4] == 'X':
            if n[4] == {} and type(n[4]) != set:
                n[4] = buildGameTree(n[1], n[2])

            value = 0

            if isMaximizingPlayer:
                isMaximizingPlayer = False
            else:
                isMaximizingPlayer = True

            if type(n[4]) != set:
                value = minimaxTreeBuilder(n[4], n[2], isMaximizingPlayer, alpha, beta, visited)
            else:
#                 print(n[4])
                value = minimaxTreeBuilder(n, n[2], isMaximizingPlayer, alpha, beta, visited)

            bestValue = max(bestValue, -value)
            
            if isMaximizingPlayer:
                alpha = max(alpha, -value)

                if beta <= alpha:
                    nodes = nodes[:i]
                    break
            else:
                beta = max(alpha, -value)

                if beta <= alpha:
                    nodes = nodes[:i]
                    break
                                
        return bestValue
                        
            
gametree = buildGameTree(board, 0) 
start = time.time()
minimaxTreeBuilder(gametree, 0, True, -INFINITY, +INFINITY, [])
print('Time: ', time.time() - start)

returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
returned -999
return

KeyboardInterrupt: 

In [28]:
for i in range(0,5):
    if i == 3:
        continue
    print(i)

0
1
2
4


In [ ]:
def depth(d, level=1):
    if not isinstance(d, dict) or not d:
        return level
    return max(depth(d[k], level + 1) for k in d)

In [ ]:
# testTree = { 'A': {'B': {'D': [3, 5], 'E': [6, 9]}, 'C': {'F': [1, 2], 'G': [0, -1]}} }#3, 5, 6, 9, 1, 2, 0, -1 }

# minimax2(testTree, 0, True, -INFINITY, +INFINITY)

In [ ]:
# INFINITY = 999

# def minimax2(node, depth, isMaximizingPlayer, alpha, beta):
# #     print('Node', node, '\n\n\n\n')
#     if type(node) is int : #### TODO *
#         return alpha #### TODO *
    
#     if isMaximizingPlayer :
#         bestVal = -INFINITY
#         for key in node:
#             for n in node[key]:
#                 if type(n) != int:
#                     value = minimax2(node[key][n], depth+1, True, alpha, beta)
#                 else:
#                     value = minimax2(n, depth+1, True, alpha, beta)
#                 bestVal = max( bestVal, value) 
#                 alpha = max( alpha, bestVal)
#                 if beta <= alpha:
#                     break
#         return bestVal

#     else :
#         bestVal = +INFINITY 
#         for key in node:
#             for n in node[key]:
                
# #         for n in node: #### TODO *
#                 if type(n) != int:
#                     value = minimax2(node[key][n], depth+1, True, alpha, beta)
#                 else:
#                     value = minimax2(n, depth+1, True, alpha, beta)
#                 bestVal = min( bestVal, value) 
#                 beta = min( beta, bestVal)
#                 if beta <= alpha:
#                     break
#         return bestVal
    

# Checking Whether the Computer Can Win in One Move

In [ ]:
# Here is the algorithm for our Tic-Tac-Toe AI:
# First, check if we can win in the next move.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, computerLetter, i)
    if isWinner(boardCopy, computerLetter):
        return i

# Checking Whether the Player Can Win in One Move

In [ ]:
# Check if the player could win on their next move and block them.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, playerLetter, i)
    if isWinner(boardCopy, playerLetter):
        return i

# Checking Whether the Board Is Full

In [ ]:
def isBoardFull(board):
# Return True if every space on the board has been taken. Otherwise, return False.
for i in range(1, 10):
    if isSpaceFree(board, i):
        return False
    return True

# The Game Loop

In [ ]:
print('Welcome to Omega!')

In [ ]:
# Choosing the Player’s Mark and Who Goes First
playerLetter, computerLetter = inputPlayerLetter()
turn = whoGoesFirst()
print('The ' + turn + ' will go first.')
gameIsPlaying = True

In [ ]:
while gameIsPlaying:
    if turn == 'player':
        # Player's turn
        drawBoard(theBoard)
        move = getPlayerMove(theBoard)
        makeMove(theBoard, playerLetter, move)

    if isWinner(theBoard, playerLetter):
        drawBoard(theBoard)
        print('Hooray! You have won the game!')
        gameIsPlaying = False

    else:
        if isBoardFull(theBoard):
            drawBoard(theBoard)
            print('The game is a tie!')
            break

        else:
            turn = 'computer'

    else:
        # Computer's turn
        move = getComputerMove(theBoard, computerLetter)
        makeMove(theBoard, computerLetter, move)
        if isWinner(theBoard, computerLetter):
            drawBoard(theBoard)
            print('The computer has beaten you! You lose.')
            gameIsPlaying = False
        else:
            if isBoardFull(theBoard):
                drawBoard(theBoard)
                print('The game is a tie!')
                break
            else:
                turn = 'player'

# Asking the Player to Play Again

In [ ]:
print('Do you want to play again? (yes or no)')
if not input().lower().startswith('y'):
    break

In [ ]:
# def printBoard(board):
#     line = ''
#     space = 0
#     for y in range(0, N):
#         for x in range(-N, N*2):
#             try:
#                 line += Hex[(x, y, -x-y)]
#             except:
#                 line += '_'
        
#     return line
# printBoard(Hex)